# Tutorial 2: Multi-topology REXEE (MT-REXEE) simulations

The tutorial will be released soon!

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/wehs7661/ensemble_md.git/HEAD?labpath=docs%2Fexamples%2Ftutorial_2%2FMT_REXEE.ipynb)

In this tutorial, we will demonstrate how one can use the different command-line interfaces (CLIs) implemented in the Python package `ensemble_md` to prepare, perform, and analyze a MT-REXEE simulation to estimate the solvation free energy of a toy molecule composed of 4 interaction sites. For a more comprehensive understanding, we strongly recommend reading our [documentation](https://ensemble-md.readthedocs.io/en/latest/simulations.html) on launching MT-REXEE simulations before starting this tutorial. For the theory behind the MT-REXEE method, we recommend reading our [JCTC paper](https://pubs.acs.org/doi/abs/10.1021/acs.jctc.4c01268) or the [theory section](https://ensemble-md.readthedocs.io/en/add_tutorials/theory.html) of our documentation.

To run this tutorial, you will need different setups depending on your computing environment:

- You can simply click the "launch binder" badge above to run this tutorial on Binder without installing anything.
- If you prefer to run this tutorial locally, either on you laptop, workstation, or an HPC cluster, you will need to install the following software:
    - [GROMACS](http://manual.gromacs.org/documentation/current/install-guide/index.html) (version 2025.0 or later)
        - We recommend using GROMACS version 2025.0 or later in order to utilize the MDP option `init_histogram_counts` which will significantly increase the speed with which weights equilibrate.
    - [ensemble_md](https://ensemble-md.readthedocs.io/en/latest/getting_started.html#installation) (version 1.1.0 or later)

Additionally, you will need to have MPI launcher commands (e.g., `mpirun` or `mpiexec`) installed on your system to run the GROMACS simulations in parallel. You will need at least four CPU cores to run the simulation in this tutorial. (You can execute `nproc` in the terminal to check the number of CPU cores on your system.)

Notably, this tutorial assumes that you understand the basics of the expanded ensemble (EE) method and relevant simulation parameters in GROMACS. If not, we recommend reading the [this tutorial](https://weitsehsu.com/course/advanced_sampling/exe/) or the following GROMACS documentation pages:

- [A very brief description about the expanded ensemble method](https://manual.gromacs.org/current/reference-manual/algorithms/expanded-ensemble.html)
- Relevant MDP parameters in GROMACS, especially the section of [free energy calcualtion](https://manual.gromacs.org/current/user-guide/mdp-options.html#free-energy-calculations) and the section of [expanded ensemble calculations](https://manual.gromacs.org/current/user-guide/mdp-options.html#expanded-ensemble-calculations).

## 0. Background knowledge: Recommended Simulation Protocol

As in tutorial 1, this tutorial will only discuss the fixed-weight MT-REXEE simulation. Below we discuss a recommended protocol for getting to the fixed weight simulation stage.

1. Determine the optimal $\lambda$ state spacing with independent expanded ensemble(EE) calulcations.
    * We recommend that one performs independent short EE simulations for at least one transformation for which you plan to run MT-REXEE. In general if the tranformations are varying a similar number of heavy atoms then the same $\lambda$ state spacing will be sufficent for each. 
    * If your set of simulations contains significantly different transformations then we recommend ensuring that your $\lambda$ state spacing produces sufficent state overlap for each significantly different transformation.
2. Perform weight-updating MT-REXEE simulations.
    * The only difference between the set-up for weight-updating and fixed-weight MT-REXEE simulations is a few MDP parameters
        * Weight-updating: `lmc_weights_equil = wl-delta` vs Fixed-weight: `lmc_weights_equil = yes`
        * Weight-updating: `lmc_stats = wang-landau` vs Fixed-weight: `lmc_stats = no`
        * The following MDP options should only be used for weight-updating simulations
            * `wl_scale = ...`
            * `wl_ration = ...`
            * `init_wl_delta = ...`
            * `weight_equil_wl_delta = ...`
3. Perform fixed-weight MT-REXEE simulation.
    * Extract the equilibrated weights from the last iteration of the weight-updating MT-REXEE simulation and provide these with the MDP option `init_lambda_weights = ...`.
    * This step will be discussed in detail below.

## 1. Preparing simulation inputs for a REXEE simulation

As mentioned in our documentation, running a REXEE simulation at least requires the following four input files:

- One YAML file that specifies REXEE parameters.
- `N` GROMACS GRO file of the system of interst.
- `N` GROMACS TOP file of the system of interst.
- One GROMACS MDP template for customizing MDP files for different replicas during the simulation.

The `N` is equal to the number of independent transformation you wish to perform with MT-REXEE. In the folder where this tutorial resides (`docs/examples/tutoria_2` in the repository), you should find all these necessary input files, including `params.yaml`, `A-B.gro`, `B-C.gro`, `C-D.gro`, `D-E.gro`, `E-F.gro`, `A-B.top`, `B-C.top`, `C-D.top`, `D-E.top`, `E-F.top`, and `expanded.mdp`. Note that the file `example_outputs.zip` is not relevant until the third section ([Analyzing a REXEE simulation](#3.-Analyzing-a-REXEE-simulation)).

In [ ]:
!ls

The MDP template `expanded.mdp` can be inspected that it adopts common/typical settings for a fixed-weight EE simulation, since here our goal is to set up a fixed-weight REXEE simulation. In the MDP template, we define 7 alchemical intermediate states for each of the 5 transformations being performed here. To enable correct parsing of the MDP file the number of states for each transformation should be equilivalent, but the values may be different. In this example we use the same $\lambda$ state spacing for each transformation.

During the REXEE simulation, customized MDP files will be generated for different replicas such that the MDP file for each replica only considers the set of states that the replica is constrained to. Notably, the weights specified via the MDP parameter `init_lambda_weights` were obtained from a weight-updating MT-REXEE simulation. 

Now, with these three GROMACS inputs prepared, let's briefly review the input YAML file that specifies MT-REXEE parameters. (For detailed information about all possible parameters that can be specified in the YAML file, please refer to our [documentation](https://ensemble-md.readthedocs.io/en/latest/simulations.html#input-yaml-parameters).)

In [ ]:
!cat params.yaml

As a minimum working example, this YAML file only specifies the compulsory simulation parameters and adopts default values for all optional parameters, execpt that `runtime_args` is specified to run each EE replica using one thread (`-nt 1`) for a reasonable simulation performance. Notably, Binder should have 72 cores by default (as can be checked by `nproc`), but using `-nt 1` is much faster than using other values for this toy system. If you are not using Binder but your own local machine or an HPC cluster, you may want to try out other `-nt` values to ensure reasonable performance.

In our case, we will run our a REXEE simulation composed of 4 replicas, with each replica performed with one thread (`-nt 1`). With the total number of states as 9 and a state shift of 1, this means that replicas 0, 1, 2, 3 are constrained to sampling states 0-5, 1-6, 2-7, and 3-8, respectively. The simulation will attempt to swap coordinates between replicas every 500 integration steps, and perform 5 iterations in total. (Note that generally we need a much larger number of iterations. Here we use a very small value just for demosntration purposes.)

If you are interested in trying out other numbers of replicas or state shifts given 9 alchemical intermediate states, you can use the CLI `explore_REXEE` to enumerate all possible REXEE configurations.